In [1]:
import os

In [2]:
%pwd

'/workspaces/newsite/IOT-Network-Intrusion-Detection-System/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/workspaces/newsite/IOT-Network-Intrusion-Detection-System'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/RCgit123/IOT-Network-Intrusion-Detection-System.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="RCgit123"
os.environ["MLFLOW_TRACKING_PASSWORD"]="764cd30f3c26c6f3bed4f4c08104c49d12732937"


In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    root_dir: Path
    model_path: Path
    test_data_path: Path
    lables_data_path: Path
    metric_file_name: Path
    mlflow_uri: str




In [7]:
from IOT_NIDS.constants import *
from IOT_NIDS.utils.common import read_yaml, create_directories, save_json

In [8]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH):
        self.config = read_yaml(config_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_evaluation_config(self) -> EvaluationConfig:
           

        config = self.config.model_evaluation
        

        create_directories([config.root_dir])

        evaluation_config = EvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            lables_data_path = config.lables_data_path,
            model_path = config.model_path,
            metric_file_name = config.metric_file_name,
            mlflow_uri="https://dagshub.com/RCgit123/IOT-Network-Intrusion-Detection-System.mlflow",
           
        )

        return evaluation_config

In [19]:
import tensorflow as tf
from pathlib import Path
import os
import numpy as np
import mlflow
import mlflow.keras
import pandas as pd
from urllib.parse import urlparse
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.preprocessing import MinMaxScaler



class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config=config

    def eval_metrics(self, actual, pred):    
        accuracy = accuracy_score(actual, pred)
        precision_per_class = precision_score(actual, pred, average=None)
        recall_per_class = recall_score(actual, pred, average=None)
        f1_score_per_class = f1_score(actual, pred, average=None)
        return accuracy, precision_per_class, recall_per_class, f1_score_per_class
    

    def log_into_mlflow(self):
        min_max_scaler = MinMaxScaler()
        x_test = pd.read_csv(self.config.test_data_path)
        y_test= pd.read_csv(self.config.lables_data_path)
        model = tf.keras.models.load_model(self.config.model_path)

       
        x_test = min_max_scaler.fit_transform(x_test)

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_lable = model.predict(x_test)
            predicted_ids=np.argmax(predicted_lable, axis=1)

            (acc, pre, recal,f1sc) = self.eval_metrics(y_test, predicted_ids)
            
        
            # Saving metrics as local
            scores = {"acc": acc, "pre": pre.tolist(), "recl": recal.tolist(), "f1s": f1sc.tolist()}
            
            # for key, value in scores.items():
            #     if isinstance(value, np.ndarray):
            #         scores[key] = value.tolist()

            save_json(path=Path(self.config.metric_file_name), data=scores)
            
            mlflow.log_metric("accuracy", np.mean(acc))
            mlflow.log_metric("precision", np.mean(pre))
            mlflow.log_metric("recall", np.mean(recal))
            mlflow.log_metric("f1_score", np.mean(f1sc))
 # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="GRUModel")
            else:
                mlflow.sklearn.log_model(model, "model")

    
 

In [20]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_evaluation_config()
    model_evaluation_config = Evaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2023-12-07 06:22:04,592: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-07 06:22:04,594: INFO: common: created directory at: artifacts]
[2023-12-07 06:22:04,594: INFO: common: created directory at: artifacts/model_evaluation]


2023-12-07 06:22:05.031633: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-12-07 06:22:05.032887: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-12-07 06:22:05.033789: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

12083/12083 [==============================] - 50s 4ms/step
[2023-12-07 06:22:59,074: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


/home/codespace/.conda/envs/nids/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2023/12/07 06:23:04 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpc6sjfgha/model/model.pkl, flavor: sklearn), fall back to return ['scikit-learn==1.3.2', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback.
Registered model 'GRUModel' already exists. Creating a new version of this model...
2023/12/07 06:23:10 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: GRUModel, version 2
Created version '2' of model 'GRUModel'.
